# Colab Code

from google.colab import drive
drive.mount('/content/drive')

import shutil
import os
 
FILE_NAME = "Data.zip"
 
def copy_zip_file(src_path, dest_dir):
 
    zip_filename = os.path.basename(src_path)
    dest_path = os.path.join(dest_dir, zip_filename)
 
    if not os.path.exists(src_path):
        print(f"Error: The file '{src_path}' does not exist.")
        return
 
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
 
    shutil.copy2(src_path, dest_path)
    print(f"'{zip_filename}' has been copied to '{dest_dir}'.")
 
source_path = "/content/drive/MyDrive/" + FILE_NAME
destination_directory = "/content"
 
copy_zip_file(source_path, destination_directory)

!unzip Data.zip

tf.config.list_physical_devices('GPU')

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")

print(f"-> {tf.config.list_physical_devices('GPU')}")

import tensorflow as tf

print(tf.__version__)

In [1]:
# imports 
import os
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import tensorflow as tf
from collections import Counter
import random
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imblearn.under_sampling import RandomUnderSampler
import shutil

# Read data

In [3]:
# Define Paths
dataset_dir = r"C:\Users\diogo\OneDrive\Documents\MEIA\AAUT2IA\data"  # Root folder containing class folders
# Adjust class names to match actual folder names
classes = ["Mild Dementia", "Moderate Dementia", "Non Demented", "Very mild Dementia"]

# Load Data with Correct Folder Names
image_paths, labels = [], []
for class_label, class_name in enumerate(classes):
    class_dir = os.path.join(dataset_dir, class_name)
    if not os.path.exists(class_dir):
        print(f"Error: Folder {class_dir} does not exist.")
        continue
    files = glob.glob(f"{class_dir}/*.jpg")  # Adjust extension if needed
    print(f"Class: {class_name}, Files Found: {len(files)}")  # Debug: Count files
    for file_path in files:
        image_paths.append(file_path)
        labels.append(class_label)

# Proceed with the pipeline if files are found
if len(image_paths) == 0:
    raise ValueError("No images found. Check dataset folder names or file paths.")

Class: Mild Dementia, Files Found: 5002
Class: Moderate Dementia, Files Found: 488
Class: Non Demented, Files Found: 67222
Class: Very mild Dementia, Files Found: 13709


In [4]:

def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [128, 128])
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    return image

# Load Dataset
def load_dataset(image_paths, labels):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda x, y: (preprocess_image(x), y))
    return dataset



In [5]:
# Apply preprocessing so all datasets contain image tensors (not paths)
preprocessed_dataset = load_dataset(image_paths, labels)

# Split by class
class_2_dataset = preprocessed_dataset.filter(lambda img, label: tf.equal(label, 2)).shuffle(buffer_size=1000).take(15_000)
class_3_dataset = preprocessed_dataset.filter(lambda img, label: tf.equal(label, 3))
class_0_dataset = preprocessed_dataset.filter(lambda img, label: tf.equal(label, 0))
class_1_dataset = preprocessed_dataset.filter(lambda img, label: tf.equal(label, 1))

# Combine the datasets
final_dataset = (class_2_dataset
                 .concatenate(class_3_dataset)
                 .concatenate(class_0_dataset)
                 .concatenate(class_1_dataset))

In [6]:

label_counts = Counter()

# Loop through dataset
for _, lbl in final_dataset:
    class_index = int(lbl.numpy())  # Convert one-hot to class index
    label_counts[class_index] += 1

# Print class distribution
print("Final dataset class distribution:")
for label, count in sorted(label_counts.items()):
    print(f"Class {label}: {count} images")


Final dataset class distribution:
Class 0: 5002 images
Class 1: 488 images
Class 2: 15000 images
Class 3: 13709 images


In [7]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np

def save_dataset_to_folder(dataset, folder_path, class_names):
    os.makedirs(folder_path, exist_ok=True)

    for class_name in class_names:
        class_dir = os.path.join(folder_path, class_name)
        os.makedirs(class_dir, exist_ok=True)

    counter = [0] * len(class_names)  # To count how many images per class

    for image, label in dataset:
        # If label is one-hot encoded, convert to integer
        if tf.rank(label) > 0:
            label = tf.argmax(label, axis=-1)

        print(label)
        label = int(label.numpy())
        class_name = class_names[label]

        # Convert tensor to numpy and save as PNG
        image_np = image.numpy()
        if image_np.dtype != 'uint8':
            image_np = (image_np * 255).astype('uint8')  # normalize if needed

        img = Image.fromarray(image_np)
        img_path = os.path.join(folder_path, class_name, f'{counter[label]}.jpg')
        img.save(img_path)

        counter[label] += 1



In [8]:

save_dataset_to_folder(final_dataset, "final_dataset", classes)


tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype

In [9]:
# Define Paths
dataset_dir = "final_dataset"  # Root folder containing class folders
# Adjust class names to match actual folder names
classes = ["Mild Dementia", "Moderate Dementia", "Non Demented", "Very mild Dementia"]

# Load Data with Correct Folder Names
image_paths, labels = [], []
for class_label, class_name in enumerate(classes):
    class_dir = os.path.join(dataset_dir, class_name)
    if not os.path.exists(class_dir):
        print(f"Error: Folder {class_dir} does not exist.")
        continue
    files = glob.glob(f"{class_dir}/*.jpg")  # Adjust extension if needed
    print(f"Class: {class_name}, Files Found: {len(files)}")  # Debug: Count files
    for file_path in files:
        image_paths.append(file_path)
        labels.append(class_label)

# Proceed with the pipeline if files are found
if len(image_paths) == 0:
    raise ValueError("No images found. Check dataset folder names or file paths.")

Class: Mild Dementia, Files Found: 5002
Class: Moderate Dementia, Files Found: 488
Class: Non Demented, Files Found: 15000
Class: Very mild Dementia, Files Found: 13709


In [10]:
# Split data into training, testing and validation
train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.3, random_state=42,stratify=labels) # suffle by default and straity labels 
test_paths, val_paths, test_labels, val_labels = train_test_split(test_paths, test_labels, test_size=0.5, random_state=42,stratify=test_labels) # suffle by default and straity labels
# to keep the same class distribution

In [11]:
train_dataset = load_dataset(train_paths, train_labels)
val_dataset = load_dataset(val_paths, val_labels)
test_dataset = load_dataset(test_paths, test_labels)

In [12]:
save_dataset_to_folder(train_dataset, "train_dataset", classes)
save_dataset_to_folder(val_dataset, "val_dataset", classes)
save_dataset_to_folder(test_dataset, "test_dataset", classes)

tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype

# Balance the dataset

In [13]:
# Define Paths
dataset_dir = "train_dataset"  # Root folder containing class folders
# Adjust class names to match actual folder names
classes = ["Mild Dementia", "Moderate Dementia", "Non Demented", "Very mild Dementia"]

# Load Data with Correct Folder Names
image_paths, labels = [], []
for class_label, class_name in enumerate(classes):
    class_dir = os.path.join(dataset_dir, class_name)
    if not os.path.exists(class_dir):
        print(f"Error: Folder {class_dir} does not exist.")
        continue
    files = glob.glob(f"{class_dir}/*.jpg")  # Adjust extension if needed
    print(f"Class: {class_name}, Files Found: {len(files)}")  # Debug: Count files
    for file_path in files:
        image_paths.append(file_path)
        labels.append(class_label)

# Proceed with the pipeline if files are found
if len(image_paths) == 0:
    raise ValueError("No images found. Check dataset folder names or file paths.")

Class: Mild Dementia, Files Found: 3501
Class: Moderate Dementia, Files Found: 342
Class: Non Demented, Files Found: 10500
Class: Very mild Dementia, Files Found: 9596


In [15]:
# -------------------------
# Setup GPU memory growth (optional but recommended)
# -------------------------
import tensorflow as tf
import numpy as np
from imblearn.over_sampling import SMOTE
from PIL import Image
import os
import gc

# Enable memory growth on GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
    except:
        pass

# -------------------------
# Image Preprocessing
# -------------------------
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [128, 128])
    image = tf.cast(image, tf.float16) / 255.0  # Use float16 to save memory
    return image

def load_and_preprocess_images(image_paths):
    images = [preprocess_image(path) for path in image_paths]
    return tf.stack(images)  # (n_samples, 128, 128, 3)

# -------------------------
# Load & Prepare Dataset
# -------------------------
X = load_and_preprocess_images(image_paths)
X = tf.reshape(X, [X.shape[0], -1])  # Flatten for SMOTE
X = X.numpy()
y = np.array(labels)

# -------------------------
# Apply SMOTE
# -------------------------
print("Before SMOTE:", dict(zip(*np.unique(y, return_counts=True))))
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)
print("After SMOTE:", dict(zip(*np.unique(y_balanced, return_counts=True))))

# -------------------------
# Reshape back to images
# -------------------------
X_balanced = X_balanced.reshape(-1, 128, 128, 3)
X_balanced = np.clip(X_balanced, 0, 1)  # Ensure values in [0, 1]

# -------------------------
# Create TF Dataset using generator (memory-safe)
# -------------------------
def smote_data_generator(X, y):
    for i in range(len(X)):
        yield X[i], y[i]

smote_dataset = tf.data.Dataset.from_generator(
    lambda: smote_data_generator(X_balanced.astype(np.float32), y_balanced),
    output_signature=(
        tf.TensorSpec(shape=(128, 128, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int64)
    )
)

# -------------------------
# Batched Saving Function
# -------------------------
def save_dataset_to_folder_batched(dataset, folder_path, class_names, batch_size=128):
    os.makedirs(folder_path, exist_ok=True)
    for class_name in class_names:
        os.makedirs(os.path.join(folder_path, class_name), exist_ok=True)

    counter = [0] * len(class_names)
    dataset = dataset.batch(batch_size)

    for batch in dataset:
        images, labels = batch
        for image, label in zip(images, labels):
            label = int(label.numpy())
            class_name = class_names[label]

            image_np = (image.numpy() * 255).astype('uint8')
            img = Image.fromarray(image_np)
            img_path = os.path.join(folder_path, class_name, f'{counter[label]}.jpg')
            img.save(img_path)
            counter[label] += 1

        del images, labels
        gc.collect()

# -------------------------
# Save to Disk
# -------------------------
classes = ['Non Demented', 'Very mild Dementia', 'Mild Dementia', 'Moderate Dementia']
save_dataset_to_folder_batched(smote_dataset, "smote_dataset", classes, batch_size=128)


Before SMOTE: {0: 3501, 1: 342, 2: 10500, 3: 9596}


C:\Users\diogo\miniconda3\envs\tf210\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


After SMOTE: {0: 10500, 1: 10500, 2: 10500, 3: 10500}


In [16]:
import tensorflow as tf
import numpy as np
from imblearn.over_sampling import SMOTE

# Image preprocessing functions
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [128, 128])
    image = tf.cast(image, tf.float16) / 255.0
    return image

def load_and_preprocess_images(image_paths):
    images = [preprocess_image(path) for path in image_paths]
    return tf.stack(images)

# Load and preprocess image data
X = load_and_preprocess_images(image_paths)  # shape: (n_samples, 128, 128, 3)
X = tf.reshape(X, [X.shape[0], -1])  # Flatten to (n_samples, features)
X = X.numpy()  # Convert to NumPy for SMOTE

# Convert labels to NumPy array
y = np.array(labels)

# Apply SMOTE directly
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

print("Before SMOTE:", dict(zip(*np.unique(y, return_counts=True))))
print("After SMOTE:", dict(zip(*np.unique(y_balanced, return_counts=True))))


C:\Users\diogo\miniconda3\envs\tf210\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Before SMOTE: {0: 3501, 1: 342, 2: 10500, 3: 9596}
After SMOTE: {0: 10500, 1: 10500, 2: 10500, 3: 10500}


In [18]:
# Reshape flattened vectors back to (128, 128, 3)
X_balanced = X_balanced.reshape(-1, 128, 128, 3)

# Optional: make sure values are in [0, 1] range (if SMOTE messed with them)
X_balanced = np.clip(X_balanced, 0, 1)

# Use a generator instead of from_tensor_slices to avoid memory issues
def smote_generator():
    for x, y in zip(X_balanced.astype(np.float32), y_balanced):
        yield x, y

# Create the TensorFlow dataset using a generator
smote_dataset = tf.data.Dataset.from_generator(
    smote_generator,
    output_signature=(
        tf.TensorSpec(shape=(128, 128, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int64)
    )
)


In [19]:
def save_dataset_to_folder_batched(dataset, folder_path, class_names, batch_size=128):
    import gc

    os.makedirs(folder_path, exist_ok=True)
    for class_name in class_names:
        os.makedirs(os.path.join(folder_path, class_name), exist_ok=True)

    counter = [0] * len(class_names)

    dataset = dataset.batch(batch_size)

    for batch in dataset:
        images, labels = batch
        for image, label in zip(images, labels):
            label = int(label.numpy())
            class_name = class_names[label]

            image_np = (image.numpy() * 255).astype('uint8')
            img = Image.fromarray(image_np)

            img_path = os.path.join(folder_path, class_name, f'{counter[label]}.jpg')
            img.save(img_path)
            counter[label] += 1

        # 🧹 Clear memory after each batch
        del images, labels
        gc.collect()


save_dataset_to_folder_batched(smote_dataset, "smote_dataset", classes, batch_size=128)


# Data Agumentation

In [20]:
# Define Paths
dataset_dir = "smote_dataset"  # Root folder containing class folders
# Adjust class names to match actual folder names
classes = ["Mild Dementia", "Moderate Dementia", "Non Demented", "Very mild Dementia"]

# Load Data with Correct Folder Names
image_paths, labels = [], []
for class_label, class_name in enumerate(classes):
    class_dir = os.path.join(dataset_dir, class_name)
    if not os.path.exists(class_dir):
        print(f"Error: Folder {class_dir} does not exist.")
        continue
    files = glob.glob(f"{class_dir}/*.jpg")  # Adjust extension if needed
    print(f"Class: {class_name}, Files Found: {len(files)}")  # Debug: Count files
    for file_path in files:
        image_paths.append(file_path)
        labels.append(class_label)

# Proceed with the pipeline if files are found
if len(image_paths) == 0:
    raise ValueError("No images found. Check dataset folder names or file paths.")

Class: Mild Dementia, Files Found: 10500
Class: Moderate Dementia, Files Found: 10500
Class: Non Demented, Files Found: 10500
Class: Very mild Dementia, Files Found: 10500


In [21]:
path_train = "smote_dataset"
path_val = "val_dataset"
#Rescale data and create data generator instances
val_datagenerator = ImageDataGenerator(rescale=1/255.)
train_datagenerator_augmentation = ImageDataGenerator(rescale = 1/255.,
                                                      rotation_range=20, #rotate the image
                                                      zoom_range = 0.2,#zoom the image
                                                      width_shift_range=0.2, #shift the image horizontally
                                                      height_shift_range=0.2, #shift the image vertically
                                                      horizontal_flip=True, #flip the image on horizontal axis
                                                      vertical_flip=True, #flip the image on vertical axis
                                                      shear_range = 0.2) #Shear the image



In [22]:
val_data = val_datagenerator.flow_from_directory(path_val,
                                                     target_size=(128,128),
                                                     batch_size=32,
                                                     class_mode='categorical'
                                                    )
train_data_augmented = train_datagenerator_augmentation.flow_from_directory(path_train,
                                                                            target_size=(128,128),
                                                                            batch_size=32,
                                                                            class_mode='categorical',
                                                                            shuffle=True)

Found 5130 images belonging to 4 classes.
Found 42000 images belonging to 4 classes.


In [23]:
# Define the CNN model
def create_cnn(num_classes=4):
    model = keras.Sequential([
        # Convolutional Block 1
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
        layers.MaxPooling2D((2,2)),
        layers.BatchNormalization(),
        
        # Convolutional Block 2
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.BatchNormalization(),
        
        # Convolutional Block 3
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.BatchNormalization(),
        
        # Flatten & Dense Layers
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),  # Reduce overfitting
        layers.Dense(num_classes, activation='softmax')  # Output layer
    ])

    # Compile the model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        # loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model



In [24]:
# Create the model
cnn_model = create_cnn()

# Print model summary
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 63, 63, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                        

In [ ]:
# Train the model

cnn_model.fit(train_data_augmented, validation_data=val_data, epochs=300)



Epoch 1/300
1313/1313 [==============================] - 100s 72ms/step - loss: 1.3481 - accuracy: 0.3710 - val_loss: 1.4848 - val_accuracy: 0.4117
Epoch 2/300
1313/1313 [==============================] - 85s 65ms/step - loss: 1.1546 - accuracy: 0.4431 - val_loss: 6.9623 - val_accuracy: 0.0713
Epoch 3/300
1313/1313 [==============================] - 85s 65ms/step - loss: 1.0662 - accuracy: 0.5054 - val_loss: 3.1981 - val_accuracy: 0.0811
Epoch 4/300
1313/1313 [==============================] - 85s 64ms/step - loss: 0.9730 - accuracy: 0.5643 - val_loss: 10.6704 - val_accuracy: 0.0614
Epoch 5/300
1313/1313 [==============================] - 85s 65ms/step - loss: 0.8474 - accuracy: 0.6267 - val_loss: 3.8203 - val_accuracy: 0.1567
Epoch 6/300
1313/1313 [==============================] - 86s 66ms/step - loss: 0.7671 - accuracy: 0.6694 - val_loss: 14.4068 - val_accuracy: 0.0347
Epoch 7/300
1313/1313 [==============================] - 86s 66ms/step - loss: 0.7172 - accuracy: 0.6945 - val_loss

In [ ]:
# Save the Model
cnn_model.save("alzheimers_detection_model_FJSL.h5")

In [ ]:
def plot_loss_curves(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    epochs = range(len(history.history['loss']))

    # Plot loss
    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

In [ ]:
plot_loss_curves(cnn_model.history)

In [ ]:
# Import model

from tensorflow.keras.models import load_model
# Load the model
cnn_model = load_model("Modelos/FJSL_Alzheimer_Classification_V2.h5")

In [ ]:
# Define Paths
dataset_dir = "val_dataset"  # Root folder containing class folders
# Adjust class names to match actual folder names
classes = ["Mild Dementia", "Moderate Dementia", "Non Demented", "Very mild Dementia"]

# Load Data with Correct Folder Names
image_paths, labels = [], []
for class_label, class_name in enumerate(classes):
    class_dir = os.path.join(dataset_dir, class_name)
    if not os.path.exists(class_dir):
        print(f"Error: Folder {class_dir} does not exist.")
        continue
    files = glob.glob(f"{class_dir}/*.jpg")  # Adjust extension if needed
    print(f"Class: {class_name}, Files Found: {len(files)}")  # Debug: Count files
    for file_path in files:
        image_paths.append(file_path)
        labels.append(class_label)

# Proceed with the pipeline if files are found
if len(image_paths) == 0:
    raise ValueError("No images found. Check dataset folder names or file paths.")

In [ ]:
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)  # Read the image from the path
    image = tf.image.decode_jpeg(image, channels=3)  # Decode the image to RGB
    image = tf.image.resize(image, [128, 128])  # Resize to (128, 128)
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    return image

# Load and preprocess a list of images
def load_and_preprocess_images(image_paths):
    # Preprocess all images and store them in a list
    images = [preprocess_image(image_path) for image_path in image_paths]
    
    # Stack the images into a single batch (shape: (batch_size, 128, 128, 3))
    images_batch = tf.stack(images)
    
    return images_batch


test_data = load_and_preprocess_images(image_paths)

# Convert labels to NumPy array
labels = np.array(labels)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming y_true is the actual labels (e.g., integer labels)
y_true = labels  # Actual labels for the test set
predictions = cnn_model.predict(test_data)
y_pred = np.argmax(predictions, axis=1)  # Convert probabilities to class labels

print("True Labels:", y_true)
print("Predicted Labels:", y_pred)

# Classification Report
class_names = classes  # List of class names
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion Matrix
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
